### This is the code to import data from the Brazilian budget execution system

#### Source of data
- For data from 2001-2013: Brazilian budget system. Available at https://www.siop.planejamento.gov.br/modulo/login/index.html#/. Data collected in Jan 25th, 2024. It contains annual data from 2001 to 2023, but we'll use only data from 2001-2013.
- For data from 2014-2024: Brazilian "Tesouro Gerencial" System. Data manually obtained from Brazilian Ministry of Economy by email request. Data collected in February, 2024. It contains monthly data from 2014 to 2024.

The parameters to subset the budget data are the following:
- Function:
    Label 9 - Social Security,
    Label 10 - Health,
    Label 12 - Education.

- Expenditure category group
    Label 4 and 5 - Investments and financial reversals, 
    Label 1 - Personnel.

- Classification based on the impact on primary result - types of expenses
    Label 1 - Mandatory, 
    Labels 2 and 3 - Discretionary, 
    Labels 6, 7, 8, and 9 - Amendments.

#### Packages
- Package Pandas (2.2). (2024). [Python]. https://pandas.pydata.org/

In [1]:
# Importing the libraries
import pandas as pd
import useful_functions as uf


In [2]:
# Read in the data from 2000 to 2023
df_budget_raw_00_23 = pd.read_excel('../data/BUDGET_raw_2000-2023_25.01.2024.xlsx')
# Read in the data from 2014 to 2024
df_budget_raw_14_24 = pd.read_excel('../data/BUDGET_raw_2014-2024_feb_2024.xlsx')


In [3]:
# Firstly, let's work with data from 2000-2013
df_budget_raw_01_13 = df_budget_raw_00_23[(df_budget_raw_00_23['Ano'] >= 2001) & (df_budget_raw_00_23['Ano'] <= 2013)].copy()

# Rename columns
df_budget_raw_01_13.rename(columns={'Ano': 'year',
                                    'Função': 'function', 'Grupo de Despesa': 'group', 
                                    'Resultado Primário': 'type', 'Empenhado': 'spent_value',
                                    'Dotação Inicial': 'initial_value'}, 
                                    inplace=True)

# Extract codes for each column
df_budget_raw_01_13['function_code'] = df_budget_raw_01_13['function'].str.split(' ', n = 1).str[0].astype(int)
df_budget_raw_01_13['group_code'] = df_budget_raw_01_13['group'].str.split(' ', n = 1).str[0].astype(int)
df_budget_raw_01_13['type_code'] = df_budget_raw_01_13['type'].str.split(' ', n = 1).str[0].astype(int)

# Reorder columns and save the new dataframe as df_budget_raw_1. 
df_budget_01_13 = df_budget_raw_01_13.reindex(columns=['year', 'function', 'function_code', 'group', 'group_code', 'type', 'type_code', 'initial_value', 'spent_value'])

# Adjust the values format
for value in ['initial_value', 'spent_value']:
    df_budget_01_13.loc[:, value] = df_budget_01_13[value].str.replace('.', '').str.replace(',', '.').astype(float)

# Add date column
df_budget_01_13['date'] = pd.to_datetime(df_budget_01_13['year'], format='%Y').dt.strftime('%Y-%m-%d')

# Reorganize columns to bring the 'date' column to the first position
df_budget_01_13.insert(0, 'date', df_budget_01_13.pop('date'))

# Drop the original 'year' column
df_budget_01_13.drop(columns=['year'], inplace=True)

# Convert labels to categorical data types
df_budget_01_13['function_code'] = df_budget_01_13['function_code'].astype(str).str.lstrip('0').astype('category')
df_budget_01_13['group_code'] = df_budget_01_13['group_code'].astype('category')
df_budget_01_13['type_code'] = df_budget_01_13['type_code'].astype('category')

# Adjust the 'function' column to remove leading zeros
df_budget_01_13['function'] = df_budget_01_13['function'].astype(str).str.lstrip('0')

# Convert 'initial_value', 'spent_value' to float
df_budget_01_13['initial_value'] = df_budget_01_13['initial_value'].astype(float)
df_budget_01_13['spent_value'] = df_budget_01_13['spent_value'].astype(float)

# Since we don't have monthly data, change the day of the index to the last day of each year, since the data corresponds to the end of each year
# Set the index to be the column 'date'
df_budget_01_13.set_index('date', inplace=True)
# Change the index to the last month of the year
df_budget_01_13.index = pd.to_datetime(df_budget_01_13.index) + pd.DateOffset(month=12)
# Reset the index
df_budget_01_13.reset_index(inplace=True)

# replace the values for january 1st of each year for 0 on variables 'initial_value' and 'spent_value'
df_budget_01_13.loc[df_budget_01_13['date'].dt.month == 1, ['initial_value', 'spent_value']] = 0



C:\Users\augus\AppData\Local\Temp\ipykernel_30768\1432614061.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_budget_01_13.loc[:, value] = df_budget_01_13[value].str.replace('.', '').str.replace(',', '.').astype(float)
C:\Users\augus\AppData\Local\Temp\ipykernel_30768\1432614061.py:22: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_budget_01_13.loc[:, value] = df_budget_01_13[value].str.replace('.', '').str.replace(',', '.').astype(float)
C:\Users\augus\AppData\Local\Temp\ipykernel_30768\1432614061.py:49: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  df_bu

In [4]:
# Let's define the expenditure variables as zero in the first they of each year.
# Create a copy of the dataframe
df_budget_copy = df_budget_01_13.copy()

# Replace the day in the date from yyyy-12-01 to yyyy-01-01
df_budget_copy['date'] = df_budget_copy['date'].apply(lambda x: x.replace(day=1, month=1) if x.month == 12 else x)
# Replace the values for 'initial_value' and 'spent_value' to 0 for the first day of each year
df_budget_copy[['spent_value', 'initial_value']] = 0

# Concatenate the original and zero value DataFrames
df_budget_01_13 = pd.concat([df_budget_01_13, df_budget_copy])

# Sort the DataFrame by date
df_budget_01_13.sort_values('date', inplace=True)


In [5]:
# Now let's adjust data from 2014-2024
df_budget_14_24 = df_budget_raw_14_24.copy()

# Dictionary to map Portuguese month abbreviations to numbers
month_mapping = {
    'JAN': '01', 'FEV': '02', 'MAR': '03', 'ABR': '04',
    'MAI': '05', 'JUN': '06','JUL': '07', 'AGO': '08',
    'SET': '09', 'OUT': '10', 'NOV': '11', 'DEZ': '12'}

# Function to convert date from 'FEV/2014' format to '2014-02' format
def convert_portuguese_date(date):
    month_abbr, year = date.split('/')
    month_number = month_mapping[month_abbr.upper()]
    return f'{year}-{month_number}'

# Apply the function to convert dates
df_budget_14_24['date'] = df_budget_14_24['date'].apply(convert_portuguese_date)

# Convert to datetime
df_budget_14_24['date'] = pd.to_datetime(df_budget_14_24['date'], format='%Y-%m')

# Let's drop the columns that are not useful for our analysis
df_budget_14_24.drop(columns=['budget_final', 'budget_paid'], inplace=True)

# Drop observations where "tc" column values are "G". It means that the expense was unpredictable or extraordinary.
df_budget_14_24 = df_budget_14_24[df_budget_14_24['tc'] != 'G']

# Group by some columns
df_budget_14_24 = df_budget_14_24.groupby(['date', 'funcao_descr', 'funcao', 'gnd_descr', 'gnd', 'rp_descr', 'rp'], as_index=False).agg({'budget_initial': 'sum', 'budget_commited': 'sum'})

# Define a mapping dictionary to rename the columns and match the names of the two DataFrames
column_mapping = {
    'date': 'date', 'funcao_descr': 'function', 'funcao': 'function_code', 
    'gnd_descr': 'group','gnd': 'group_code', 'rp_descr': 'type', 'rp': 'type_code', 
    'budget_initial': 'initial_value', 'budget_commited': 'spent_value'}

# Rename the columns of the DataFrame using the mapping dictionary
df_budget_14_24.rename(columns=column_mapping, inplace=True)

# Convert string values to lowercase
df_budget_14_24['function'] = df_budget_14_24['function'].str.title()
df_budget_14_24['group'] = df_budget_14_24['group'].str.title()
df_budget_14_24['type'] = df_budget_14_24['type'].str.title()

# Convert labels to categorical data types
df_budget_14_24['function_code'] = df_budget_14_24['function_code'].astype(str).str.lstrip('0').astype('category')
df_budget_14_24['group_code'] = df_budget_14_24['group_code'].astype('category')
df_budget_14_24['type_code'] = df_budget_14_24['type_code'].astype('category')

# Modify columns "function", "group", and "type"
df_budget_14_24['function'] = df_budget_14_24['function_code'].astype(str) + ' - ' + df_budget_14_24['function']
df_budget_14_24['group'] = df_budget_14_24['group_code'].astype(str) + ' - ' + df_budget_14_24['group']
df_budget_14_24['type'] = df_budget_14_24['type_code'].astype(str) + ' - ' + df_budget_14_24['type']

# Convert 'initial_value', 'spent_value' to float
df_budget_14_24['initial_value'] = df_budget_14_24['initial_value'].astype(float)
df_budget_14_24['spent_value'] = df_budget_14_24['spent_value'].astype(float)

In [6]:
# Concatenate the two DataFrames
df_budget_raw = pd.concat([df_budget_01_13, df_budget_14_24], ignore_index=True)

# Convert date column to datetime
df_budget_raw['date'] = pd.to_datetime(df_budget_raw['date'], format='%Y-%m-%d')


In [7]:
# Mask the data to keep only the desired functions, groups and types
desired_functions = ['9', '10', '12'] #Function 9 - Pension, Function 10 - Health, Function 12 - Education
desired_groups = [1, 4, 5] # Group 1 - Personal, Group 4 - Investiments and group 5 - Financial changes
desired_types = [1, 2, 3, 6, 7, 8, 9] # Group 1 - Personal, Group 4 - Investiments and group 5 - Financial changes

# Filter by function
df_function = df_budget_raw[df_budget_raw['function_code'].isin(desired_functions)].copy()[['date', 'function', 'function_code', 'initial_value', 'spent_value']]
df_function.rename(columns={'function_code': 'code'}, inplace=True)

# Filter by group
df_group = df_budget_raw[df_budget_raw['group_code'].isin(desired_groups)].copy()[['date', 'group', 'group_code', 'initial_value', 'spent_value']]
df_group.rename(columns={'group_code': 'code'}, inplace=True)

# Filter by type
df_type = df_budget_raw[df_budget_raw['type_code'].isin(desired_types)].copy()[['date', 'type', 'type_code', 'initial_value', 'spent_value']]
df_type.rename(columns={'type_code': 'code'}, inplace=True)


In [8]:
# Group df_function by date and sum the values
df_step= df_function.groupby(['date', 'code']).sum().reset_index()

# Create a new dictionary to store the filtered dataframes
filtered_dfs = {}

# Loop through the types of values to create the filtered dataframes
for i, value in enumerate(['initial_value', 'spent_value']):
    
    filtered_dfs[i]  = df_step[['date', 'code', value]].copy().pivot(index='date', columns='code', values=value) # filter and pivot the dataframe and them store it in the dictionary
    
    for column in filtered_dfs[i].columns:
        filtered_dfs[i].rename(columns={column: f"BUDGET_fun_{column}_{value}"}, inplace=True) # rename the columns to avoid confusion

df_function = pd.concat(filtered_dfs.values(), axis=1) # concatenate the dataframes from the dictionary




C:\Users\augus\AppData\Local\Temp\ipykernel_30768\2886678715.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_step= df_function.groupby(['date', 'code']).sum().reset_index()


In [9]:
# Group df_group by date and sum the values
df_step= df_group.groupby(['date', 'code']).sum().reset_index()

# Create a new dictionary to store the filtered dataframes
filtered_dfs = {}

# Loop through the types of values to create the filtered dataframes
for i, value in enumerate(['initial_value', 'spent_value']):
    
    filtered_dfs[i]  = df_step[['date', 'code', value]].copy().pivot(index='date', columns='code', values=value) # filter and pivot the dataframe and them store it in the dictionary
    
    for column in filtered_dfs[i].columns:
        filtered_dfs[i].rename(columns={column: f"BUDGET_group_{column}_{value}"}, inplace=True) # rename the columns to avoid confusion

# Concatenate the dataframes from the dictionary
df_group = pd.concat(filtered_dfs.values(), axis=1)

# Create new columns to store the sum of the initial values from the groups 4 and 5
df_group['BUDGET_group_invest_initial_value'] = df_group[['BUDGET_group_4_initial_value', 'BUDGET_group_5_initial_value']].sum(axis=1)
df_group.drop(['BUDGET_group_4_initial_value', 'BUDGET_group_5_initial_value'], axis=1, inplace=True)

# Create new columns to store the sum of the spent values from the groups 4 and 5
df_group['BUDGET_group_invest_spent_value'] = df_group[['BUDGET_group_4_spent_value', 'BUDGET_group_5_spent_value']].sum(axis=1)
df_group.drop(['BUDGET_group_4_spent_value', 'BUDGET_group_5_spent_value'], axis=1, inplace=True)

# Rename columns
df_group.rename(columns={'BUDGET_group_1_initial_value': 'BUDGET_group_personal_initial_value',
                        'BUDGET_group_1_spent_value': 'BUDGET_group_personal_spent_value'}, inplace=True)



C:\Users\augus\AppData\Local\Temp\ipykernel_30768\1175113068.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_step= df_group.groupby(['date', 'code']).sum().reset_index()


In [10]:
# Group df_type by date and sum the values
df_step= df_type.groupby(['date', 'code']).sum().reset_index()

# Create a new dictionary to store the filtered dataframes
filtered_dfs = {}

#   Loop through the types of values to create the filtered dataframes
for i, value in enumerate(['initial_value', 'spent_value']):
    
    filtered_dfs[i]  = df_step[['date', 'code', value]].copy().pivot(index='date', columns='code', values=value) # filter and pivot the dataframe and them store it in the dictionary
    
    for column in filtered_dfs[i].columns:
        filtered_dfs[i].rename(columns={column: f"BUDGET_type_{column}_{value}"}, inplace=True) # rename the columns to avoid confusion

# Concatenate the dataframes from the dictionary
df_type = pd.concat(filtered_dfs.values(), axis=1)

#Let's generate new variables based on the type of RP
# Create new columns to store the sum of the initial values from the types 6, 7, 8 and 9
df_type['BUDGET_type_amendments_initial_value'] = df_type[['BUDGET_type_6_initial_value', 'BUDGET_type_7_initial_value', 'BUDGET_type_8_initial_value', 'BUDGET_type_9_initial_value']].sum(axis=1)
df_type['BUDGET_type_disc_initial_value'] = df_type[['BUDGET_type_2_initial_value', 'BUDGET_type_3_initial_value']].sum(axis=1)
df_type.drop(['BUDGET_type_6_initial_value', 'BUDGET_type_7_initial_value', 'BUDGET_type_8_initial_value', 'BUDGET_type_9_initial_value', 'BUDGET_type_2_initial_value', 'BUDGET_type_3_initial_value'], axis=1, inplace=True)

# Create new columns to store the sum of the spent values from the types 6, 7, 8 and 9
df_type['BUDGET_type_amendments_spent_value'] = df_type[['BUDGET_type_6_spent_value', 'BUDGET_type_7_spent_value', 'BUDGET_type_8_spent_value', 'BUDGET_type_9_spent_value']].sum(axis=1)
df_type['BUDGET_type_disc_spent_value'] = df_type[['BUDGET_type_2_spent_value', 'BUDGET_type_3_spent_value']].sum(axis=1)
df_type.drop(['BUDGET_type_6_spent_value', 'BUDGET_type_7_spent_value', 'BUDGET_type_8_spent_value', 'BUDGET_type_9_spent_value', 'BUDGET_type_2_spent_value', 'BUDGET_type_3_spent_value'], axis=1, inplace=True)

# Rename columns
df_type.rename(columns={'BUDGET_type_1_initial_value': 'BUDGET_type_mandatory_initial_value',
                        'BUDGET_type_1_spent_value': 'BUDGET_type_mandatory_spent_value'}, inplace=True)



C:\Users\augus\AppData\Local\Temp\ipykernel_30768\3669022178.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_step= df_type.groupby(['date', 'code']).sum().reset_index()


In [11]:
# Concatenate the dataframes
df_budget = pd.concat([df_function, df_group, df_type], axis=1)

# Since the initial variables contains several 0 observations, they will not be useful. Let's drop it.
columns_to_drop = [col for col in df_budget.columns if 'initial' in col]
df_budget = df_budget.drop(columns=columns_to_drop)

# Convert time index to datetime and adjust its format.
df_budget.index = pd.to_datetime(df_budget.index, format='%Y%m%d')

In [12]:
# # Convert the index to the desired format
df_budget.index = df_budget.index.strftime('%Y-%m')

# Adjust the index name
df_budget.rename_axis('Date', inplace=True)

# Export the dataframe to an excel file
df_budget.to_csv('../data/df_budget.csv')
